# Model assessment

To assess the performance of the QSPR models, there are two methods available in `QSPRpred`
from the [`assessment_methods`](https://cddleiden.github.io/QSPRpred/docs/api/qsprpred.models.html#module-qsprpred.models.assessment_methods) module:
* CrossValAssessor: Performs cross validation on a dataset
* TestSetAssessor: Performs predictions on a test set

In this notebook, we will demonstrate how to use these methods.
Let's start by loading the data and creating the model we want to assess.

In [1]:
import os
import pandas as pd
from IPython.display import display
from qsprpred.data.data import QSPRDataset
from qsprpred.data.utils.descriptorsets import FingerprintSet
from qsprpred.data.utils.descriptorcalculator import MoleculeDescriptorsCalculator
from qsprpred.data.utils.datasplitters import RandomSplit

os.makedirs("../../tutorial_output/data", exist_ok=True)

dataset = QSPRDataset.fromTableFile(
  	filename='../../tutorial_data/A2A_LIGANDS.tsv', 
  	store_dir="../../tutorial_output/data",
  	name="AssessmentTutorialDataset",
  	target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
  	random_state=42
)

display(dataset.getDF())

# Calculate MorganFP features
feature_calculator = MoleculeDescriptorsCalculator(desc_sets = [FingerprintSet(fingerprint_type="MorganFP", radius=3, nBits=2048)])

# calculate compound features and split dataset into train and test
dataset.prepareDataset(
    split=RandomSplit(test_fraction=0.2, dataset=dataset),
    feature_calculators=[feature_calculator],
    recalculate_features=True,
)

dataset.getDF().head()

from qsprpred.models.sklearn import SklearnModel
from sklearn.neighbors import KNeighborsRegressor

os.makedirs("../../tutorial_output/models", exist_ok=True)

# This is an SKlearn model, so we will initialize it with the SklearnModel class
model = SklearnModel(
    base_dir = '../../tutorial_output/models',
    data = dataset,
    alg = KNeighborsRegressor,
    name = 'AssessmentTutorialModel',
)

,SMILES,pchembl_value_Mean,Year,QSPRID
QSPRID,,,,
AssessmentTutorialDataset_0000,Cc1nn(-c2cc(NC(=O)CCN(C)C)nc(-c3ccc(C)o3)n2)c(...,8.68,2008.0,AssessmentTutorialDataset_0000
AssessmentTutorialDataset_0001,Nc1c(C(=O)Nc2ccc([N+](=O)[O-])cc2)sc2c1cc1CCCC...,4.82,2010.0,AssessmentTutorialDataset_0001
AssessmentTutorialDataset_0002,O=C(Nc1nc2ncccc2n2c(=O)n(-c3ccccc3)nc12)c1ccccc1,5.65,2009.0,AssessmentTutorialDataset_0002
AssessmentTutorialDataset_0003,CNC(=O)C12CC1C(n1cnc3c1nc(C#CCCCCC(=O)OC)nc3NC...,5.45,2009.0,AssessmentTutorialDataset_0003
AssessmentTutorialDataset_0004,CCCn1c(=O)c2c(nc3cc(OC)ccn32)n(CCCNC(=O)c2ccc(...,5.20,2019.0,AssessmentTutorialDataset_0004
...,...,...,...,...
AssessmentTutorialDataset_4077,CNc1ncc(C(=O)NCc2ccc(OC)cc2)c2nc(-c3ccco3)nn12,7.09,2018.0,AssessmentTutorialDataset_4077
AssessmentTutorialDataset_4078,Nc1nc(-c2ccco2)c2ncn(C(=O)NCCc3ccccc3)c2n1,8.22,2008.0,AssessmentTutorialDataset_4078
AssessmentTutorialDataset_4079,Nc1nc(Nc2ccc(F)cc2)nc(CSc2nnc(N)s2)n1,4.89,2010.0,AssessmentTutorialDataset_4079


## Cross-validation
Now we will assess our model using cross-validation. We will use the `CrossValAssessor` class for this.
The `CrossValAssessor` class takes a model as input and uses the attached dataset. The training dataset is split into a number of folds, and the model is trained on each fold. The performance of the model is then assessed on the fold that was not used for training,
and the results are stored in a file `{model_name}_cv.tsv`.

Note. if the dataset was not split into a train and a test set, the whole dataset is used for cross-validation.

To score the performance of the model, we need to provide a scoring function. This function should take the true values and the predicted values as input, and return a score. The scores are returned by the function.
TODO: Add more info about scoring functions

In [2]:
from qsprpred.models.assessment_methods import CrossValAssessor
from qsprpred.models.metrics import SklearnMetric

score_func = SklearnMetric.getMetric(name='r2')

# Create a CrossValAssessor object
CrossValAssessor(score_func)(model)


/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed

[0.6381005757810776,
 0.6470413968481483,
 0.6722435248029048,
 0.6588247607859833,
 0.5749937649418198]

Furthermore, we can specify the splitting strategy. 
By default, the dataset is split into 5 folds using a shuffle split.
You can find more information on how to split the data in the [data splitting tutorial](../data/data_splitting.ipynb).


In [3]:
from sklearn.model_selection import KFold

split = KFold(n_splits=10, shuffle=True, random_state=dataset.randomState)
CrossValAssessor(score_func, split=split)(model)

/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed

[0.6326776666707019,
 0.6679638227844381,
 0.6316441275385969,
 0.6644615008506205,
 0.6094387718918599,
 0.701911410383543,
 0.6275067773672989,
 0.6822540428310435,
 0.543230466648694,
 0.5948557924176254]

## Test set validation
To assess the performance of the model on the specified test set, we can use the `TestSetAssessor` class.
It works in a similar way as the `CrossValAssessor`, but instead of splitting the dataset into folds, it uses the complete training set to train the model, and then uses the associated test set to assess the performance of the model.
It therefore does not require a splitting strategy, but it does require a test set to have been defined in the dataset by performing an initial split during dataset preparation.
The results are stored in a file `{model_name}.ind.tsv` ("ind" from independent test set).

In [4]:
from qsprpred.models.assessment_methods import TestSetAssessor
TestSetAssessor(score_func)(model)

/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


[0.63020184117675]

## More information
These two classes are subclasses of the `ModelAssessor` class, which contains the shared functionality.
To customize the behavior of the assessors, you can create a subclass the `ModelAssessor` class.
Furthermore, the `ModelAssessor` objects can be used to assess the model performance
in hyperparameter optimization, as is shown in the [hyperparameter optimization tutorial](../../advanced/modelling/hyperparameter_optimization.ipynb).